<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_13_tokens_y_representaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 13 – Tokens y Representaciones

## Objetivos
- Comprender cómo representar texto en formato numérico.
- Comparar Bag-of-Words, TF-IDF y Embeddings.
- Implementar embeddings preentrenados.






---

### ¿Qué es NLP?

El **Procesamiento de Lenguaje Natural (NLP, por sus siglas en inglés)** es una rama de la inteligencia artificial que estudia cómo las computadoras pueden comprender, interpretar y generar lenguaje humano.
Sus aplicaciones incluyen traducción automática, chatbots, análisis de sentimientos, clasificación de texto, extracción de información, entre muchas otras.

---

### Tokens en NLP

En NLP, los *tokens* son las unidades mínimas en que se divide el texto para su análisis.

* **Tokenización tradicional**: cada palabra se considera un token (ej. “aprendiendo NLP” → \[“aprendiendo”, “NLP”]).
* **Tokenización subword**: divide palabras en fragmentos más pequeños (ej. “aprendiendo” → \[“aprend”, “iendo”]), útil para manejar vocabularios extensos y palabras desconocidas.
* **Caracteres**: en algunos sistemas, cada letra o carácter es un token.

---

### Bag-of-Words (BoW), TF-IDF y Embeddings

#### Bag-of-Words (BoW)

* Representa el texto como un vector que cuenta la frecuencia de cada palabra en un vocabulario.
* Ventajas: sencillo, interpretable.
* Limitaciones:

  * Pierde el orden de las palabras.
  * Altísima dimensionalidad (un vector por cada palabra del vocabulario).
  * No captura relaciones semánticas (ej. “gato” y “felino” aparecen como vectores totalmente distintos).

#### TF-IDF (Term Frequency – Inverse Document Frequency)

* Mejora sobre BoW ponderando cada palabra según:

  * **TF (Frecuencia de término)**: cuántas veces aparece en un documento.
  * **IDF (Frecuencia inversa de documento)**: qué tan rara es esa palabra en la colección de documentos.
* Intuición: las palabras comunes como “el” o “la” reciben menor peso, mientras que términos distintivos como “neuronas” o “transformer” reciben mayor relevancia.
* Sigue siendo esparso y no captura relaciones semánticas profundas, pero mejora la representación frente a BoW.

#### Embeddings

* Son representaciones densas y de baja dimensión, donde cada palabra (o token) se proyecta en un espacio vectorial.
* Capturan similitudes semánticas: palabras con significados cercanos quedan más próximas en el espacio.
* Ejemplos clásicos: **Word2Vec**, **GloVe**, **FastText**.
* Ejemplo: “rey - hombre + mujer ≈ reina”.

**Importancia:** Los embeddings permiten que los modelos comprendan mejor las relaciones entre palabras, mejorando traducción automática, clasificación de texto, análisis de sentimientos, etc.

---

### Arquitectura Transformer

Los **Transformers** revolucionaron el NLP al introducir mecanismos de *atención* en lugar de depender de redes recurrentes (RNN/LSTM).

* **Self-Attention**: cada token puede “prestar atención” a todos los demás en la secuencia, capturando dependencias largas.
* **Paralelización**: al no ser recurrente, se entrena de manera más eficiente.
* **Escalabilidad**: es la base de los grandes modelos de lenguaje modernos.

---

### Modelos basados en Transformers: BERT y RoBERTa

#### BERT (*Bidirectional Encoder Representations from Transformers*)

* Propuesto por Google (2018).
* Pre-entrenado en grandes corpus con dos tareas principales:

  * *Masked Language Modeling (MLM)*: predecir palabras ocultas en una oración.
  * *Next Sentence Prediction (NSP)*: predecir si una oración sigue a otra.
* Es bidireccional: tiene en cuenta el contexto a izquierda y derecha de cada token.
* Se adapta bien a múltiples tareas mediante *fine-tuning* (clasificación, preguntas y respuestas, etc.).

#### RoBERTa (*Robustly Optimized BERT Approach*)

* Propuesto por Facebook AI.
* Mejora sobre BERT al:

  * Entrenar con más datos y durante más tiempo.
  * Usar secuencias más largas.
  * Eliminar la tarea NSP, que resultaba poco útil.
* Generalmente supera a BERT en benchmarks de NLP.

---

✅ **Resumen actualizado:**
El **NLP** busca enseñar a las máquinas a comprender el lenguaje humano. Para representarlo, se empezó con enfoques simples como **Bag-of-Words** y **TF-IDF**, que cuentan frecuencias pero no capturan significado profundo. Los **embeddings** aportaron representaciones densas y semánticas, abriendo paso a modelos más poderosos. Finalmente, los **Transformers**, con arquitecturas como **BERT** y **RoBERTa**, lograron un entendimiento contextual bidireccional, revolucionando el estado del arte en múltiples tareas de NLP.

---


In [ ]:
# Ejemplo de Clasificador (LogReg/Naive Bayes) con BoW/TF-IDF.

from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar dataset AG News
dataset = load_dataset("ag_news")
X_train = dataset["train"]["text"]
y_train = dataset["train"]["label"]
X_test = dataset["test"]["text"]
y_test = dataset["test"]["label"]

# 2. Vectorizadores (BoW y TF-IDF)
bow_vectorizer = CountVectorizer(max_features=5000, stop_words="english")
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")

X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 3. Modelos a comparar
models = {
    "LogReg_BoW": (LogisticRegression(max_iter=1000), X_train_bow, X_test_bow),
    "NaiveBayes_BoW": (MultinomialNB(), X_train_bow, X_test_bow),
    "LogReg_TFIDF": (LogisticRegression(max_iter=1000), X_train_tfidf, X_test_tfidf),
    "NaiveBayes_TFIDF": (MultinomialNB(), X_train_tfidf, X_test_tfidf),
}

results = []

# 4. Entrenar y evaluar
for name, (model, Xtr, Xte) in models.items():
    model.fit(Xtr, y_train)
    y_pred = model.predict(Xte)
    acc = accuracy_score(y_test, y_pred)
    results.append({"Modelo": name, "Accuracy": acc})

    # --- Matriz de confusión ---
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=dataset["train"].features["label"].names,
                yticklabels=dataset["train"].features["label"].names)
    plt.title(f"Matriz de Confusión - {name}")
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.show()

# 5. Tabla comparativa de accuracies
df_results = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)
print("\nResultados comparativos:\n", df_results)



In [ ]:
# Ejemplo de Bag-of-Words vs embeddings

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

# Ejemplo de oraciones
sentences = [
    "El gato duerme en la silla",
    "Un felino descansa en la silla",
]

# --- Bag of Words ---
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(sentences).toarray()

# --- Embeddings con modelo preentrenado ---
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
embeddings = model.encode(sentences)

# --- Similaridades ---
bow_sim = cosine_similarity([bow_matrix[0]], [bow_matrix[1]])[0][0]
embed_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

import pandas as pd
df = pd.DataFrame({
    "Método": ["Bag-of-Words", "Embeddings"],
    "Representación (dim)": [str(bow_matrix.shape[1]), str(embeddings.shape[1])],
    "Similitud coseno": [bow_sim, embed_sim]
})

df


**Checklist:**

1. Cargar dataset (IMDB o Yelp).
2. Mostrar 5 reseñas aleatorias y discutir su tono.
3. Escribir 2 reseñas propias (positiva y negativa).
4. Tokenizar esas reseñas con Hugging Face `AutoTokenizer`.
5. Medir longitud de tokens y compararla con el texto original.
6. Visualizar tokens generados (IDs y palabras).


---

## 1. Cargar dataset y explorar

In [ ]:
from datasets import load_dataset

# IMDB reviews (50k reseñas)
dataset = load_dataset("imdb")
print(dataset)

# Ejemplo
print(dataset["train"][0])

---

## 2. Explorar reseñas y escribir ejemplos propios

In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import pipeline

# Convierte el split 'train' a un DataFrame de pandas
df = pd.DataFrame(dataset["train"])

# Configura pandas para mostrar el contenido completo de las columnas
pd.set_option('display.max_colwidth', None)

# Filtra las reseñas que tienen menos de 140 caracteres
# (El dataset IMDB tiene textos largos, ajustamos el filtro si es necesario)
short_reviews_df = df[df['text'].str.len() < 140]

# Toma una muestra aleatoria reproducible de 5 reseñas cortas
sample_of_short_reviews = short_reviews_df.sample(n=5, random_state=42)
print("Muestra de 6 reseñas cortas seleccionada.")

# Traducción del texto

try:
    # 6. Carga el pipeline de traducción (descargará el modelo la primera vez)
    print("\nCargando el modelo de traducción (Helsinki-NLP/opus-mt-en-es)...")
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
    print("Modelo de traducción cargado. ✅")

    # 7. Extrae la lista de textos en inglés del DataFrame
    texts_to_translate = sample_of_short_reviews['text'].tolist()

    # 8. Traduce la lista de textos
    print("\nTraduciendo textos...")
    translated_texts = translator(texts_to_translate)
    print("Traducción completada.")

    # 9. Añade la traducción como una nueva columna al DataFrame
    sample_of_short_reviews['texto_traducido'] = [t['translation_text'] for t in translated_texts]

    # 10. Muestra el DataFrame final con ambas columnas
    print("\n--- Resultado Final: Reseñas Originales y Traducidas ---")
    print(sample_of_short_reviews[['text', 'texto_traducido', 'label']])

except Exception as e:
    print(f"\nHa ocurrido un error: {e}")
    print("Asegúrate de tener todas las librerías instaladas.")




---

## 3. Tokenizar reseñas

In [ ]:
from transformers import AutoTokenizer

my_reviews = [
    "Fui a ver 'Ecos del Silencio' este fin de semana sin muchas expectativas, y salí de la sala completamente maravillado.",
    "Tenía muchas ganas de ver 'Misión Cifrada', pero lamentablemente ha sido una gran decepción."
]

# RoBERTa base en español, con tokenizador BPE estándar
tokenizer = AutoTokenizer.from_pretrained("bertin-project/bertin-roberta-base-spanish")

print("✅ Tokenizador RoBERTa en español cargado correctamente.")

tokens = tokenizer(my_reviews, padding=True, truncation=True)
print("\n--- Resultado de la Tokenización ---")
print(tokens)

---

## 4. Análisis de tokens

* Comparar número de palabras vs tokens.
* Visualizar palabras → IDs.

In [ ]:
# --- Código para comparar palabras vs. tokens ---

for i, review in enumerate(my_reviews):
    print(f"--- RESEÑA #{i+1} ---")
    print(f"Texto original: \"{review}\"")

    # 1. Conteo de palabras (método simple: dividir por espacios)
    word_count = len(review.split())
    print(f"🔹 Número de Palabras: {word_count}")

    # 2. Conteo de tokens (usando el tokenizador)
    # Tokenizamos el texto y obtenemos los IDs de los tokens
    token_ids = tokenizer.encode(review)

    # Contamos el número total de tokens
    token_count = len(token_ids)
    print(f"🔸 Número de Tokens: {token_count}")

    # Para visualizar, convertimos los IDs de vuelta a tokens
    tokens_list = tokenizer.convert_ids_to_tokens(token_ids)
    print(f"   Tokens generados: {tokens_list}\n")


---

## Preguntas de discusión

### 1. ¿Qué ventajas tienen los embeddings frente a bag-of-words?

Los **embeddings** (incrustaciones de palabras) representan un avance fundamental sobre el modelo **bag-of-words** (BoW) porque capturan el **significado semántico** y el **contexto** de las palabras, algo que BoW es incapaz de hacer.

El modelo **bag-of-words** únicamente registra la frecuencia de las palabras en un texto, pero ignora por completo su orden y la relación que tienen entre sí. Para BoW, las frases "el perro persigue al gato" y "el gato persigue al perro" son muy similares, aunque su significado es opuesto. Además, trata palabras como "rey" y "reina" como dos conceptos totalmente independientes y sin ninguna relación.

Los **embeddings**, en cambio, mapean palabras a vectores de números de tal manera que las palabras con significados similares tienen vectores cercanos en un espacio multidimensional.

Las principales ventajas son:

* **Captura de Relaciones Semánticas:** Los embeddings sitúan palabras como "rey" y "reina" cerca una de la otra. Incluso pueden capturar relaciones analógicas, como la famosa `vector('rey') - vector('hombre') + vector('mujer') ≈ vector('reina')`.
* **Eficiencia Dimensional:** Mientras que BoW crea vectores muy largos (uno por cada palabra del vocabulario) y dispersos (llenos de ceros), los embeddings son vectores densos y de menor dimensión (ej. 300 dimensiones vs. 50,000 de BoW), lo que los hace computacionalmente más eficientes.
* **Generalización:** Un modelo pre-entrenado con embeddings (como Word2Vec o GloVe) ya "sabe" que "excelente" y "fantástico" son similares, incluso si en tu set de datos de entrenamiento nunca aparecen en el mismo contexto. BoW no puede hacer esta generalización.

En resumen, pasar de bag-of-words a embeddings es como pasar de un simple conteo de palabras a una verdadera comprensión de su significado y de cómo se relacionan entre sí.

---

### 2. ¿Qué tipo de reseñas serían más difíciles de clasificar?

Los modelos de clasificación de texto, incluso los más avanzados, tienen dificultades con reseñas que requieren una comprensión profunda del lenguaje humano, el contexto y el conocimiento del mundo. Las más difíciles son:

* **Sarcasmo e Ironía:** Son el mayor desafío. Una reseña como *"Claro, me encantó esperar 40 minutos por un café frío. Una experiencia fantástica."* utiliza palabras positivas ("encantó", "fantástica") para expresar un sentimiento fuertemente negativo. El modelo, al leer literalmente las palabras, puede clasificarla erróneamente.

* **Reseñas Mixtas o Ambiguas:** Aquellas que contienen tanto elementos positivos como negativos son difíciles de encasillar en una sola categoría. Por ejemplo: *"La comida era deliciosa y el ambiente muy bueno, pero el servicio fue pésimo y arruinó la noche."* ¿Es una reseña positiva o negativa? Depende del peso que se le dé a cada aspecto.

* **Lenguaje Comparativo:** Reseñas que evalúan un producto en relación con otro. *"Es mucho mejor que el modelo anterior, aunque sigue sin estar a la altura de la competencia."* La clasificación depende de un punto de referencia que el modelo puede no conocer.

* **Falta de Contexto o Conocimiento del Mundo:** Frases que requieren conocimiento externo. Por ejemplo, *"Este producto es el 'New Coke' de los videojuegos."* Para entender que esto es una crítica muy negativa, el modelo necesitaría saber sobre el famoso fracaso comercial de Coca-Cola en los años 80.

* **Textos muy cortos o con jerga:** Reseñas como *"meh"* o *"equis"* son difíciles porque contienen muy poca información contextual. De igual manera, el uso de jerga muy específica de un nicho puede confundir a un modelo que no fue entrenado con ella.